### **To-Do List:**

1. Look into which other parts of the code I can turn into functions/modules. 
   
2. Find where I can meaningfully test with pytest (see [here](https://realpython.com/python-testing/) and [here](https://coderefinery.github.io/testing/)).
   
 --> **Read articles, how much of this should I immediately go into? Installed pytest, confirmed it works with:
 "pytest -v tests/test.py", looking for practical things to test.**

3. Make package pip-installable, as per the instructions [here](https://aaltoscicomp.github.io/python-for-scicomp/packaging/). 
  
  --> **Added pyproject.toml, made it Git-ignorable, haven't tested local pip install yet.**

4. More detailed README and module documentation, based on recommendations [here](https://coderefinery.github.io/documentation/). **Can GitHub Copilot help?**

5. Add LICENSE file (https://github.com/coderefinery/social-coding/blob/main/licensing-and-cakes.md).

**Warnings:**

- FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.concat(dfs, axis=0, sort=True))

--> **Seems like different types are being concatenated and this warning is triggered, but it's fine for now. Track future Pandas developments.**

- c:\Users\lordm\Desktop\Projects\code_mqs\.venv\lib\site-packages\ax\core\data.py
  
FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object. args["df"] = pd.read_json(args["df"]["value"], dtype=False)

--> In data.py, I replaced this:

        if "df" in args and not isinstance(args["df"], pd.DataFrame):
            # NOTE: Need dtype=False, otherwise infers arm_names like
            # "4_1" should be int 41.
            args["df"] = pd.read_json(args["df"]["value"], dtype=False)

with this:

        import io
        if "df" in args and not isinstance(args["df"], pd.DataFrame):
            # Wrap the JSON string in a StringIO object and use pd.io.json.read_json
            args["df"] = pd.io.json.read_json(io.StringIO(args["df"]["value"]), dtype=False)
        return extract_init_args(args=args, class_=cls)

--> **Fixed.**

### Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties
from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep
from ax.modelbridge.registry import Models

### Training Data and Parameters

In [8]:
unique_parameters = ["tungstate_concentration", "current_density", "deposition_time", "pH"] 
# temperature avoided, kept constant at 25 C

# EXAMPLE, here use actual data
X_train = np.array([[0.05, 10, 500, 5],
               [0.05, 50, 300, 6],
               [0.1, 10, 300, 7],
               [0.1, 10, 600, 8],
               
               [0.1, 50, 600, 7.5],
               [0.1, 100, 600, 10],
               [0.1, 50, 400, 6.5],
               [0.1, 30, 600, 8.5],
               [0.15, 10, 600, 9.5],
               [0.15, 50, 300, 9.5],
               
               # NEW OPTIMIZATION DATA    
               ])

# Convert to dataframe
X_train = pd.DataFrame(X_train, columns=unique_parameters)
X_train

,tungstate_concentration,current_density,deposition_time,pH
0,0.05,10.0,500.0,5.0
1,0.05,50.0,300.0,6.0
2,0.10,10.0,300.0,7.0
3,0.10,10.0,600.0,8.0
4,0.10,50.0,600.0,7.5
5,0.10,100.0,600.0,10.0
6,0.10,50.0,400.0,6.5
7,0.10,30.0,600.0,8.5
8,0.15,10.0,600.0,9.5
9,0.15,50.0,300.0,9.5


In [9]:
# Number of data points
n_train = X_train.shape[0]

unique_objectives = ["overpotential", "overpotential_slope"]

# EXAMPLE, here use actual observations
y_train = np.array([[-358,0.00015], 
                    [-319,0.000066], 
                    [-377,0.0001], 
                    [-319,-0.000518],
                    [-286,0.00008], 
                    [-312,0.000029], 
                    [-309,-0.000057], 
                    [-290,0.001656],
                    [-329,0.000131], 
                    [-305,-0.000064],

                    # NEW OPTIMIZATION DATA                    
                    ])

# Convert to dataframe
y_train = pd.DataFrame(y_train, columns=unique_objectives)
y_train

,overpotential,overpotential_slope
0,-358.0,0.000150
1,-319.0,0.000066
2,-377.0,0.000100
3,-319.0,-0.000518
4,-286.0,0.000080
5,-312.0,0.000029
6,-309.0,-0.000057
7,-290.0,0.001656
8,-329.0,0.000131
9,-305.0,-0.000064


In [10]:
parameters=[
        {
            "name": "tungstate_concentration",
            "type": "range",
            "bounds": [0.05, 0.2],
        },
        {
            "name": "current_density",
            "type": "range",
            "bounds": [5, 125],
        },
        {
            "name": "deposition_time",
            "type": "range",
            "bounds": [60, 600],
        },
        {
            "name": "pH",
            "type": "range",
            "bounds": [5, 10],
        }
        ]

### Generation Strategy

In [12]:
# Specify a custom generation strategy - Skips the pseudo-random initial suggested points 
gs = GenerationStrategy(
    steps=[
        # Quasi-random Sobol sequence along with my initial data
        # GenerationStep(
            # model=Models.SOBOL, 
            # num_trials=5,  # How many trials to produce during generation step
            # min_trials_observed=3,  # How many trials to be completed before next model
            # max_parallelism=5  # Max parallelism for this step
        # ),
        ################
        # Skips SOBOL sampling step (which is the default first step) and uses the initial data provided
        # Bayesian optimization step (requires data obtained from previous phase and learns
        # from all data available at the time of each new candidate generation call)
        GenerationStep(
            model = Models.MOO,
            # Uses qNEHVI by default, use instead of GPEI (for single) and PAREGO (for multi but worse than qNEHVI)
            # Not passing any information about the observation noise; so the model is inferring a Gaussian noise level on the observations - SingleTaskGP
            num_trials=-1,  # No limitation on how many trials should be produced from this step
            max_parallelism=3  # BATCH/Parallelism limit for this step, often lower than for Sobol
                    )
            ]
)

### Setup the experiment

In [13]:
ax_client = AxClient(generation_strategy=gs)
ax_client.create_experiment(
    name="NiW",
    parameters=parameters,
    objectives={
        "overpotential": ObjectiveProperties(minimize=False, threshold = -350), # How ambitious + what is max (min) allowable value?
        "overpotential_slope": ObjectiveProperties(minimize=False, threshold = -0.001) # Not clear, looking for >=0 if possible
    },
    overwrite_existing_experiment=False, # I am saving my data anyway, but just to make sure
    is_test = False, # Set this to False when running a REAL experiment
    choose_generation_strategy_kwargs = {"max_parallelism_override": 3} # Parallel/Batch evaluation, max batch size = 3
)

[INFO 02-09 20:26:46] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-09 20:26:46] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter tungstate_concentration. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 02-09 20:26:46] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter current_density. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 02-09 20:26:46] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter deposition_time. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parame

### Input training data (run this only the FIRST time)

Doing this manually while also specifying parameter data types, to avoid odd errors (e.g. initial data is one type, while in the dataframe it is another + need to use **iloc** function to iterate through dataframe).

In [14]:
# Initialize trial index
trial_index = 0

# Attach the training data
for i in range(n_train):
    trial_parameters = {
        param: int(value) if param in ["current_density", "deposition_time", "pH"] else float(value)
        for param, value in X_train.iloc[i, :].items()
    }
    
    # Create a dictionary with the raw data for each objective
    trial_raw_data = {obj: float(value) for obj, value in zip(unique_objectives, y_train.iloc[i])}
    
    # Attach trial and complete it
    ax_client.attach_trial(trial_parameters)
    ax_client.complete_trial(trial_index=trial_index, raw_data=trial_raw_data)
    
    # Increment trial index
    trial_index += 1

[INFO 02-09 20:26:51] ax.core.experiment: Attached custom parameterizations [{'tungstate_concentration': 0.05, 'current_density': 10, 'deposition_time': 500, 'pH': 5}] as trial 0.
[INFO 02-09 20:26:51] ax.service.ax_client: Completed trial 0 with data: {'overpotential': (-358.0, None), 'overpotential_slope': (0.00015, None)}.
[INFO 02-09 20:26:51] ax.core.experiment: Attached custom parameterizations [{'tungstate_concentration': 0.05, 'current_density': 50, 'deposition_time': 300, 'pH': 6}] as trial 1.
[INFO 02-09 20:26:51] ax.service.ax_client: Completed trial 1 with data: {'overpotential': (-319.0, None), 'overpotential_slope': (6.6e-05, None)}.
[INFO 02-09 20:26:51] ax.core.experiment: Attached custom parameterizations [{'tungstate_concentration': 0.1, 'current_density': 10, 'deposition_time': 300, 'pH': 7}] as trial 2.
[INFO 02-09 20:26:51] ax.service.ax_client: Completed trial 2 with data: {'overpotential': (-377.0, None), 'overpotential_slope': (0.0001, None)}.
[INFO 02-09 20:26:

### Run optimization and get next suggestions (repeat cell as needed)

In [15]:
for i in range(3): # generate a batch of 3 new trials
    next_experiment, trial_index = ax_client.get_next_trial() 
    print("next suggested experiment: ", next_experiment)

[INFO 02-09 20:26:57] ax.service.ax_client: Generated new trial 10 with parameters {'tungstate_concentration': 0.096083, 'current_density': 49, 'deposition_time': 600, 'pH': 9}.
c:\Users\lordm\Desktop\Projects\code_mqs\.venv\lib\site-packages\ax\core\data.py:292: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.concat(dfs, axis=0, sort=True))
[INFO 02-09 20:26:57] ax.modelbridge.torch: The observations are identical to the last set of observations used to fit the model. Skipping model fitting.


next suggested experiment:  {'tungstate_concentration': 0.09608333121237567, 'current_density': 49, 'deposition_time': 600, 'pH': 9}


[INFO 02-09 20:27:02] ax.service.ax_client: Generated new trial 11 with parameters {'tungstate_concentration': 0.135191, 'current_density': 45, 'deposition_time': 600, 'pH': 8}.
c:\Users\lordm\Desktop\Projects\code_mqs\.venv\lib\site-packages\ax\core\data.py:292: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.concat(dfs, axis=0, sort=True))
[INFO 02-09 20:27:02] ax.modelbridge.torch: The observations are identical to the last set of observations used to fit the model. Skipping model fitting.


next suggested experiment:  {'tungstate_concentration': 0.13519093174888336, 'current_density': 45, 'deposition_time': 600, 'pH': 8}


[INFO 02-09 20:27:06] ax.service.ax_client: Generated new trial 12 with parameters {'tungstate_concentration': 0.062703, 'current_density': 44, 'deposition_time': 600, 'pH': 8}.


next suggested experiment:  {'tungstate_concentration': 0.06270290109136085, 'current_density': 44, 'deposition_time': 600, 'pH': 8}


In [ ]:
# Confirm running trials
ax_client.generation_strategy.trials_as_df

### Perform lab experiments and get new objectives

In [16]:
# Complete pendings trials
# We manually set SEM = None to indicate that we know the function to be noisy, but would like Ax to estimate the noise for us.
ax_client.complete_trial(trial_index=10, raw_data={'overpotential': (-260.0, None), 'overpotential_slope': (-0.002, None)})
ax_client.complete_trial(trial_index=11, raw_data={'overpotential': (-255.0, None), 'overpotential_slope': (-0.02, None)})
ax_client.complete_trial(trial_index=12, raw_data={'overpotential': (-245.0, None), 'overpotential_slope': (-0.0004, None)})

[INFO 02-09 20:27:11] ax.service.ax_client: Completed trial 10 with data: {'overpotential': (-260.0, None), 'overpotential_slope': (-0.002, None)}.
[INFO 02-09 20:27:11] ax.service.ax_client: Completed trial 11 with data: {'overpotential': (-255.0, None), 'overpotential_slope': (-0.02, None)}.
[INFO 02-09 20:27:11] ax.service.ax_client: Completed trial 12 with data: {'overpotential': (-245.0, None), 'overpotential_slope': (-0.0004, None)}.


In [ ]:
# Confirm completed trials
ax_client.generation_strategy.trials_as_df

#### Repeat etc.

In [ ]:
for i in range(3): # generate a batch of 3 new trials
    next_experiment, trial_index = ax_client.get_next_trial() 
    print("next suggested experiment: ", next_experiment)

In [ ]:
ax_client.complete_trial(trial_index=13, raw_data={'overpotential': (-245.0, None), 'overpotential_slope': (-0.002, None)})
ax_client.complete_trial(trial_index=14, raw_data={'overpotential': (-259.0, None), 'overpotential_slope': (+0.00004, None)})
ax_client.complete_trial(trial_index=15, raw_data={'overpotential': (-269.0, None), 'overpotential_slope': (-0.0034, None)})

In [ ]:
ax_client.generation_strategy.trials_as_df

### Save/Load the experiment

In [17]:
# Save the experiment state to disk.
ax_client.save_to_json_file("ax_client_snapshot.json")  

[INFO 02-09 20:27:19] ax.service.ax_client: Saved JSON-serialized state of optimization to `ax_client_snapshot.json`.


In [18]:
# Restore the experiment state from disk.  
ax_client = AxClient.load_from_json_file("ax_client_snapshot.json") 
# For custom filepath, pass `filepath` argument.
# For any loading issues: https://github.com/facebook/Ax/issues/1565

[INFO 02-09 20:27:20] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


### Confirm model instantiation and data so far

In [19]:
print('model:', ax_client.generation_strategy.model)
ax_client.fit_model()

# ax.service.ax_client: Successfully instantiated a model for the first time. ==> ?
# Maybe run this before get_next_trial() ?

[INFO 02-09 20:33:37] ax.service.ax_client: get_model_predictions() has been called when no model is instantiated. Attempting to instantiate the model for the first time.


model: None


[INFO 02-09 20:33:38] ax.service.ax_client: Successfully instantiated a model for the first time.
[INFO 02-09 20:33:38] ax.modelbridge.torch: The observations are identical to the last set of observations used to fit the model. Skipping model fitting.


In [20]:
ax_client.experiment.fetch_data().df

,arm_name,metric_name,mean,sem,trial_index
0,0_0,overpotential,-358.000000,NaN,0
1,0_0,overpotential_slope,0.000150,NaN,0
2,1_0,overpotential,-319.000000,NaN,1
3,1_0,overpotential_slope,0.000066,NaN,1
4,2_0,overpotential,-377.000000,NaN,2
5,2_0,overpotential_slope,0.000100,NaN,2
6,3_0,overpotential,-319.000000,NaN,3
7,3_0,overpotential_slope,-0.000518,NaN,3
8,4_0,overpotential,-286.000000,NaN,4
9,4_0,overpotential_slope,0.000080,NaN,4


### Get Pareto optimal parameters

In [22]:
pareto_optimal_params = ax_client.get_pareto_optimal_parameters(use_model_predictions=True) 
# use_model_predictions=True by default, also try False?
print("Pareto optimal parameters:", pareto_optimal_params)

TypeError: AxClient.get_pareto_optimal_parameters() got an unexpected keyword argument 'transform_outcomes_and_configs'

### Plot (Posterior) Pareto front 

In [ ]:
# Plotting imports and initialization
from ax.utils.notebook.plotting import render, init_notebook_plotting
from ax.plot.pareto_utils import compute_posterior_pareto_frontier
from ax.plot.pareto_frontier import plot_pareto_frontier

In [ ]:
objectives = ax_client.experiment.optimization_config.objective.objectives
frontier = compute_posterior_pareto_frontier(
    experiment=ax_client.experiment,
    data=ax_client.experiment.fetch_data(),
    primary_objective=objectives[1].metric,
    secondary_objective=objectives[0].metric,
    absolute_metrics=["overpotential", "overpotential_slope"],
    num_points=16
)
render(plot_pareto_frontier(frontier, CI_level=0.90))

## Hypervolume: Evaluating the Quality of a Pareto Front

#### Hypervolume for specific trial

In [ ]:
# Do this for every trial - by default calculates hypervolume UP TO and INCLUDING the current trial
# Observe difference betweeen use_model_predictions=True/False

trial_indices = [15]
model_true = ax_client.get_hypervolume(trial_indices = trial_indices, use_model_predictions=True)
model_false = ax_client.get_hypervolume(trial_indices = trial_indices, use_model_predictions=False)

print(f'Hypervolume With Model Predictions: {model_true}, Without: {model_false}, For trial: {trial_indices}')

In [ ]:
# OR use this to get the optimization trace of the ENTIRE experiment i.e. all trials:
# Equivalent to calling _get_hypervolume repeatedly, with an increasing sequence of trial_indices 
# and with use_model_predictions = FALSE, though this does it more efficiently.
hypervolume_trace = ax_client.get_trace()
print(hypervolume_trace)

#### Hypervolume over iterations = Optimization trace for multi-objective 

In [ ]:
# Plot the hypervolume trace
plt.plot(range(1, len(hypervolume_trace) + 1), hypervolume_trace, label='Hypervolume', marker='o', linestyle='-', color='b')
plt.xlabel('Iteration')
plt.ylabel('Hypervolume')
plt.title('Hypervolume over Iterations')
plt.grid(True)
plt.legend()
plt.show()

## Visualizations

In [ ]:
from ax.modelbridge.cross_validation import cross_validate
from ax.plot.diagnostic import interact_cross_validation
from ax.plot.contour import interact_contour
from ax.plot.scatter import (
    interact_fitted,
    plot_objective_vs_constraints,
    tile_fitted,
)
from ax.plot.slice import plot_slice

### Cross validation

In [ ]:
model = ax_client.generation_strategy.model
cv_results = cross_validate(model, folds= -1) # default folds
render(interact_cross_validation(cv_results))

#### Contour plot

In [ ]:
# Interactive contour plots, able to switch between different parameter-objective combinations.
render(interact_contour(model, metric_name="overpotential"))

#### Tradeoff plot 

In [ ]:
render(plot_objective_vs_constraints(model, "overpotential", rel=False)) # rel = False

#### Slice plot

In [ ]:
# Slice plot, can be done for all different parameter-objective combinations.
render(plot_slice(model, "pH", "overpotential"))

## More visualizations

Automatic data conversion from the Ax dataframe to a list, for visualization purposes.

In [ ]:
# Get dataframe and drop the 'sem' column
data_df = ax_client.experiment.fetch_data().df
data_df = data_df.drop(columns=['sem'])

# Convert the DataFrame to a list of lists
data_list = []

for trial in data_df['trial_index'].unique():
    trial_data = data_df[data_df['trial_index'] == trial]
    
    # Extract data for 'overpotential' and 'overpotential_slope'
    overpotential_data = trial_data.loc[trial_data['metric_name'] == 'overpotential', 'mean'].tolist()[0]
    slope_data = trial_data.loc[trial_data['metric_name'] == 'overpotential_slope', 'mean'].tolist()[0]
    
    # Combine the data into a list of tuples
    trial_data_list = [overpotential_data, slope_data]
    
    # Extend the main list
    data_list.append(trial_data_list)

# Now data_list has the format you want
print(data_list)

### Plot by batch or by trial

In [ ]:
from src.ax_code.visualizations import plot_pareto

# Need to be manually organized by batch, see visualizations.py 
# (needed for the 'by batch' plot, doesn't matter for the rest)

# Choose the visualization type ('batch' or 'trial')
visualization_type = 'batch' 

# Call the function to plot the Pareto front
plot_pareto(data_list, visualization_type)

### Plot by tracking one of the two objectives over the trials

In [ ]:
from src.ax_code.visualizations import plot_pareto_objective

# Choose the visualization type ('value' or 'slope')
visualization_type = 'slope' 

# Call the function to plot the Pareto front based on tracking the chosen objective
plot_pareto_objective(data_list, visualization_type)